In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# 🚀 Vision-Language-Action Models: Teaching Robots to See, Understand, and Act

**Notebook 6 of 6 — World Action Models Series | Vizuara**

**Estimated time: ~45 minutes**

In the previous notebooks, we built world models that imagine the future, learned latent dynamics, and explored how agents can plan inside a learned simulator. But there has always been a missing piece: how do we close the loop from *seeing the world* and *understanding language* all the way to *physically acting*?

Today, we bring everything together. Vision-Language-Action (VLA) models are the grand convergence — world models (imagination) + large language models (language understanding) + vision encoders (perception), all fused into a single system that outputs real robot actions.

By the end of this notebook, you will build a simplified VLA that takes an image and a language command ("reach the red circle") and outputs a smooth robot action trajectory using **flow matching** — the same technique behind Physical Intelligence's pi-zero robot controller.

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://course-creator-brown.vercel.app/courses/world-action-models/practice/6/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


## Section 1 — Why Does This Matter?

Let us start with a simple observation. Over the past few years, three revolutions happened almost simultaneously:

1. **Vision models** (like ViT, DINO) learned to see the world with remarkable precision.
2. **Language models** (like GPT, LLaMA) learned to understand and reason about instructions.
3. **World models** (like JEPA, Dreamer) learned to imagine future states from experience.

For the longest time, these lived in separate worlds. Computer vision researchers built perception systems. NLP researchers built language systems. Robotics researchers built controllers. Each module was excellent on its own — but getting them to work together was a nightmare of glue code, interface mismatches, and cascading errors.

Then, in late 2024 and 2025, a breakthrough happened: researchers asked, **"What if one model could do all three?"**

**Physical Intelligence's pi-zero ($\pi_0$)** demonstrated exactly this. A single neural network takes in camera images and a language instruction like "fold the laundry" and outputs continuous robot joint actions at 50Hz — fast enough to control a real robot arm in real time. The secret ingredient? **Flow matching** for action generation.

Meanwhile, **Meta's V-JEPA 2** showed something equally stunning: a model pre-trained on internet video (no labels, no language, just watching the world) developed such deep physical understanding that it could control a robot with minimal task-specific data. It watched millions of videos of the world and, from that alone, learned enough physics to manipulate objects.

This is the frontier of AI: **one model that sees, understands language, and acts.**

In [ ]:
# Section 0: Setup and imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.collections import LineCollection
from IPython.display import HTML
import matplotlib.animation as animation
import warnings
warnings.filterwarnings('ignore')

# Reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

## Section 2 — Building Intuition (No Code)

### The Old Way: A Relay Race

Traditionally, a robot control pipeline looked like a relay race with three runners:

1. **Perception module** — takes the camera image and extracts features ("I see a red cup at position (0.3, 0.5)")
2. **Planning module** — takes those features + the language command and plans a path ("Move to (0.3, 0.5) while avoiding obstacles")
3. **Control module** — takes the plan and converts it to motor commands ("Set joint 1 to 45 degrees, joint 2 to 30 degrees...")

Each runner passes a baton to the next. The problem? **Errors accumulate at every handoff.** If the perception module misidentifies the cup's position by 2 centimeters, the planner plans to the wrong spot, and the controller faithfully executes the wrong plan. The modules cannot correct each other because information only flows in one direction.

### The VLA Way: A Single Athlete

A Vision-Language-Action model replaces the relay race with **a single athlete who sees, thinks, and moves in one fluid motion.** There is no baton-passing, no interface mismatch, no error accumulation between stages. The entire system is trained end-to-end:

$$a_{1:T} = \pi_\theta(\text{img}, \text{text})$$

One model. Image in, language in, action trajectory out. This is exactly what we want.

### Flow Matching Intuition

Now the question is: how does the model actually *generate* the action trajectory? This is where **flow matching** comes in.

Imagine you have a cloud of random dust particles floating in the air (this is noise — random vectors sampled from a Gaussian distribution). Now imagine there is an invisible force field that gently pushes each particle along a smooth path until it lands exactly where it needs to be — forming a beautiful, smooth action trajectory.

**Flow matching learns this force field.** Formally, it learns a velocity field $v_\theta(x_t, t)$ that tells you: "If you are at position $x_t$ at time $t$, move in *this* direction." Start from random noise at $t=0$, follow the velocity field step by step, and at $t=1$ you arrive at a valid action trajectory.

Think of it like water carving a river path through random terrain. The water does not teleport to its destination — it flows smoothly, and the path it carves is the trajectory.

> **Why is this better than just directly predicting the actions?** Because robot actions need to be *smooth*. A robot arm cannot teleport — it must follow a continuous path. Flow matching naturally produces smooth trajectories because the velocity field itself is smooth. Direct regression, on the other hand, tends to produce jerky, averaged-out predictions — especially when there are multiple valid ways to reach a target.

### 🤔 Think About This

"Why is it important that the action output is a smooth trajectory rather than discrete commands?"

Consider a robot pouring water from a kettle. If the actions are discrete ("move left", "tilt", "stop"), the motion is jerky and the water splashes everywhere. But if the actions are a smooth, continuous trajectory of positions and angles, the pour is clean and controlled. Real-world physics demands smoothness — and flow matching delivers it.

## Section 3 — The Mathematics

Let us now formalize the intuitions we built above. We will break this down step by step.

### Step 1: The VLA Formulation

A Vision-Language-Action model is a policy $\pi_\theta$ that maps an observation (image) and an instruction (text) to a sequence of actions:

$$a_{1:T} = \pi_\theta(\text{img}, \text{text})$$

Here, $a_{1:T} = (a_1, a_2, \ldots, a_T)$ is a trajectory of $T$ action vectors. Each $a_t \in \mathbb{R}^d$ where $d$ is the action dimension (e.g., $d=2$ for 2D position changes).

Computationally, the image goes through a vision encoder, the text goes through a language encoder, their features are fused, and this fused representation conditions an action decoder that generates the full trajectory.

### Step 2: Flow Matching — The Velocity Field

The action decoder uses **flow matching**. The core idea is to learn a velocity field $v_\theta(x_t, t)$ that transports samples from a simple noise distribution to the data distribution:

$$v_\theta : \mathbb{R}^{T \times d} \times [0, 1] \rightarrow \mathbb{R}^{T \times d}$$

At time $t=0$, we have noise: $x_0 \sim \mathcal{N}(0, I)$. At time $t=1$, we want valid action trajectories: $x_1 \sim p_{\text{data}}$.

The velocity field tells us the direction and speed to move at each point in this "flow time."

Let us plug in some simple numbers to build intuition. Suppose our action is 1-dimensional and we have noise $x_0 = -1.5$ and target action $x_1 = 3.0$. The "ideal" velocity that would take us from noise to data in a straight line is simply $(x_1 - x_0) = 3.0 - (-1.5) = 4.5$. The model learns to predict this velocity at each interpolation point along the way.

### Step 3: The Interpolation Path

During training, we define a linear interpolation between noise and data:

$$x_t = (1 - t) \cdot x_0 + t \cdot x_1$$

At $t=0$, we are at the noise $x_0$. At $t=1$, we are at the target action $x_1$. At $t=0.5$, we are exactly halfway.

Using our numbers: at $t = 0.3$, we get $x_{0.3} = (1 - 0.3) \times (-1.5) + 0.3 \times 3.0 = 0.7 \times (-1.5) + 0.3 \times 3.0 = -1.05 + 0.9 = -0.15$. This makes sense — at 30% of the way through the flow, we are close to zero, partway between noise and the target.

### Step 4: The Training Loss

The model learns by predicting the velocity at each interpolation point. The target velocity for the linear interpolation is simply:

$$u_t = x_1 - x_0$$

And the loss is:

$$\mathcal{L} = \mathbb{E}_{t \sim U(0,1),\; x_0 \sim \mathcal{N}(0,I),\; x_1 \sim p_{\text{data}}} \left[ \| v_\theta(x_t, t) - (x_1 - x_0) \|^2 \right]$$

In plain English: sample a random time $t$, sample noise $x_0$, sample a real action trajectory $x_1$, compute the interpolation point $x_t$, ask the model to predict the velocity at that point, and penalize it if the prediction differs from the true velocity $(x_1 - x_0)$.

Let us verify with our numbers. If $x_0 = -1.5$ and $x_1 = 3.0$, the target velocity is $3.0 - (-1.5) = 4.5$. If the model predicts $v_\theta = 4.2$, the loss is $(4.5 - 4.2)^2 = 0.09$. If the model predicts $4.5$ exactly, the loss is $0$. This is exactly what we want.

### Step 5: Inference via ODE Integration

At inference time, we start from noise $x_0 \sim \mathcal{N}(0, I)$ and integrate the learned velocity field using Euler steps:

$$x_{t + \Delta t} = x_t + \Delta t \cdot v_\theta(x_t, t)$$

We take $K$ steps from $t=0$ to $t=1$, with step size $\Delta t = 1/K$. The final $x_1$ is our generated action trajectory.

Using our 1D example with $K=4$ steps ($\Delta t = 0.25$): starting from $x_0 = -1.5$, if the model has learned perfectly ($v_\theta \approx 4.5$ everywhere), then after step 1: $x_{0.25} = -1.5 + 0.25 \times 4.5 = -0.375$. After step 2: $x_{0.5} = -0.375 + 0.25 \times 4.5 = 0.75$. After step 3: $x_{0.75} = 0.75 + 0.25 \times 4.5 = 1.875$. After step 4: $x_{1.0} = 1.875 + 0.25 \times 4.5 = 3.0$. We arrive exactly at the target. This is the power of flow matching.

In [ ]:
# Quick demonstration: flow matching in 1D
# Let us verify our numerical example from above

x_0 = -1.5  # noise sample
x_1 = 3.0   # target action
K = 4        # number of Euler steps
dt = 1.0 / K

# True velocity (what a perfectly trained model predicts)
true_velocity = x_1 - x_0
print(f"True velocity: {true_velocity}")

# Euler integration
x_t = x_0
trajectory = [x_t]
for step in range(K):
    t = step * dt
    x_t = x_t + dt * true_velocity
    trajectory.append(x_t)
    print(f"Step {step+1}: t={t+dt:.2f}, x_t={x_t:.3f}")

print(f"\nFinal x_1 = {trajectory[-1]:.3f} (target = {x_1})")

Now let us visualize this flow in 1D to build even deeper intuition.

In [ ]:
# Visualize the 1D flow matching process
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: The interpolation path
t_vals = np.linspace(0, 1, 50)
x_t_vals = (1 - t_vals) * x_0 + t_vals * x_1
axes[0].plot(t_vals, x_t_vals, 'b-', linewidth=2)
axes[0].scatter([0, 1], [x_0, x_1], c=['red', 'green'],
                s=100, zorder=5)
axes[0].set_xlabel('Flow time t')
axes[0].set_ylabel('x_t')
axes[0].set_title('Interpolation: Noise → Action')
axes[0].legend(['Path', 'Noise (t=0)', 'Target (t=1)'])

# Plot 2: The velocity field (constant for linear flow)
axes[1].barh([0], [true_velocity], color='purple', height=0.3)
axes[1].set_xlabel('Velocity')
axes[1].set_title(f'Target Velocity = {true_velocity}')
axes[1].set_yticks([])

# Plot 3: Euler integration steps
steps_t = np.linspace(0, 1, K + 1)
axes[2].plot(steps_t, trajectory, 'ro-', linewidth=2, markersize=8)
axes[2].plot(t_vals, x_t_vals, 'b--', alpha=0.3)
axes[2].set_xlabel('Flow time t')
axes[2].set_ylabel('x_t')
axes[2].set_title(f'Euler Integration ({K} steps)')

plt.tight_layout()
plt.show()

## Section 4 — Let Us Build It

Now we have all the mathematical machinery. Let us build a complete (simplified) Vision-Language-Action model for a 2D reaching task.

Here is the setup:
- A "robot arm" (2-link arm in 2D) must reach a colored target
- **Observation** = an image of the scene (rendered with matplotlib)
- **Action** = a sequence of $(dx, dy)$ movements that move the end-effector
- **Language instruction** = "reach the [color] circle"

We will build every component from scratch: vision encoder, language encoder, fusion module, and flow matching action decoder.

### 4.1 — The 2D Environment

Let us create our simple environment. We will render a scene with a 2-link robot arm and a colored target circle. The robot must learn to move its end-effector to the target.

In [ ]:
# Environment constants
IMG_SIZE = 64
ARM_LINK_1 = 0.3   # length of first link
ARM_LINK_2 = 0.25  # length of second link
ARM_BASE = np.array([0.5, 0.1])  # base position (normalized)
TRAJ_LEN = 16      # action trajectory length
ACTION_DIM = 2     # (dx, dy) per step

COLORS = {
    'red': [1.0, 0.2, 0.2],
    'blue': [0.2, 0.4, 1.0],
    'green': [0.2, 0.8, 0.3],
}
COLOR_NAMES = list(COLORS.keys())

def forward_kinematics(theta1, theta2):
    """Compute end-effector position from joint angles."""
    joint = ARM_BASE + ARM_LINK_1 * np.array(
        [np.cos(theta1), np.sin(theta1)])
    end = joint + ARM_LINK_2 * np.array(
        [np.cos(theta1 + theta2), np.sin(theta1 + theta2)])
    return joint, end

In [ ]:
def render_scene(target_pos, target_color_name,
                 theta1=np.pi/3, theta2=np.pi/4,
                 trajectory=None):
    """Render the 2D scene as a numpy image array."""
    fig, ax = plt.subplots(1, 1, figsize=(2, 2), dpi=32)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_facecolor('#1a1a2e')

    # Draw target circle
    color_rgb = COLORS[target_color_name]
    circle = plt.Circle(target_pos, 0.06, color=color_rgb,
                        alpha=0.9)
    ax.add_patch(circle)

    # Draw robot arm
    joint, end = forward_kinematics(theta1, theta2)
    ax.plot([ARM_BASE[0], joint[0]], [ARM_BASE[1], joint[1]],
            'w-', linewidth=3)
    ax.plot([joint[0], end[0]], [joint[1], end[1]],
            'w-', linewidth=3)
    ax.plot(*ARM_BASE, 'wo', markersize=6)
    ax.plot(*joint, 'wo', markersize=4)
    ax.plot(*end, 'yo', markersize=5)

    # Draw trajectory if provided
    if trajectory is not None:
        traj = np.cumsum(trajectory, axis=0) + end
        ax.plot(traj[:, 0], traj[:, 1], 'y--', linewidth=1,
                alpha=0.7)

    fig.canvas.draw()
    buf = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    w, h = fig.canvas.get_width_height()
    img = buf.reshape(h, w, 4)[:, :, :3]  # drop alpha
    plt.close(fig)
    return img

Let us visualize our environment with different targets.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for idx, color_name in enumerate(COLOR_NAMES):
    target = np.array([0.3 + 0.2 * idx, 0.6 + 0.1 * idx])
    img = render_scene(target, color_name)
    axes[idx].imshow(img)
    axes[idx].set_title(f'"reach the {color_name} circle"',
                        fontsize=11)
    axes[idx].axis('off')
plt.suptitle('Our 2D Robot Environment', fontsize=14,
             fontweight='bold')
plt.tight_layout()
plt.show()

### 4.2 — Generating Training Data

Before we build the model, let us generate our training dataset. For each sample, we randomly place a colored target, compute the ideal trajectory from the arm's end-effector to the target, and render the scene image.

In [ ]:
def generate_trajectory(start_pos, target_pos, n_steps=TRAJ_LEN):
    """Generate a smooth trajectory from start to target.

    Returns (dx, dy) deltas at each step, with slight
    curvature for realism.
    """
    t = np.linspace(0, 1, n_steps + 1)
    # Add slight sinusoidal curvature
    curve = 0.03 * np.sin(np.pi * t[:-1])
    direction = target_pos - start_pos
    perp = np.array([-direction[1], direction[0]])
    perp = perp / (np.linalg.norm(perp) + 1e-8)

    positions = np.outer(t[1:], direction) + start_pos
    positions += np.outer(curve, perp)

    # Convert to deltas
    all_pos = np.vstack([start_pos[None, :], positions])
    deltas = np.diff(all_pos, axis=0)
    return deltas.astype(np.float32)

In [ ]:
def generate_dataset(n_samples=2000):
    """Generate the full training dataset."""
    images = []
    instructions = []
    trajectories = []

    theta1, theta2 = np.pi / 3, np.pi / 4
    _, end_effector = forward_kinematics(theta1, theta2)

    for i in range(n_samples):
        # Random target position (reachable area)
        tx = np.random.uniform(0.15, 0.85)
        ty = np.random.uniform(0.35, 0.90)
        target_pos = np.array([tx, ty])
        color_name = COLOR_NAMES[i % len(COLOR_NAMES)]

        # Render scene
        img = render_scene(target_pos, color_name,
                           theta1, theta2)
        # Generate trajectory
        traj = generate_trajectory(end_effector, target_pos)

        images.append(img)
        instructions.append(color_name)  # simplified encoding
        trajectories.append(traj)

    images = np.stack(images)              # (N, H, W, 3)
    trajectories = np.stack(trajectories)  # (N, T, 2)
    return images, instructions, trajectories

print("Generating training data...")
images, instructions, trajectories = generate_dataset(2000)
print(f"Images shape: {images.shape}")
print(f"Trajectories shape: {trajectories.shape}")
print(f"Sample instructions: {instructions[:6]}")

Let us visualize a few training samples with their ground truth trajectories.

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for idx in range(4):
    axes[idx].imshow(images[idx])
    # Overlay trajectory
    _, end = forward_kinematics(np.pi/3, np.pi/4)
    traj_pos = np.cumsum(trajectories[idx], axis=0) + end
    # Scale to pixel coordinates
    h, w = images[idx].shape[:2]
    px = traj_pos[:, 0] * w
    py = (1 - traj_pos[:, 1]) * h  # flip y
    axes[idx].plot(px, py, 'y.-', linewidth=1.5, markersize=3)
    axes[idx].set_title(f'"{instructions[idx]}"')
    axes[idx].axis('off')
plt.suptitle('Training Samples with Ground Truth Trajectories',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

### 4.3 — The Vision Encoder

Our vision encoder is a simple CNN that takes the scene image and encodes it into a compact feature vector. In the real $\pi_0$ system, this would be a large pre-trained vision transformer. For our 2D task, a small CNN is sufficient.

In [ ]:
class VisionEncoder(nn.Module):
    """Simple CNN that encodes a scene image into features."""

    def __init__(self, feature_dim=64):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
        )
        self.fc = nn.Linear(64, feature_dim)

    def forward(self, img):
        # img: (B, 3, H, W) normalized to [0, 1]
        x = self.conv(img)         # (B, 64, 1, 1)
        x = x.view(x.size(0), -1)  # (B, 64)
        return self.fc(x)           # (B, feature_dim)

vision_enc = VisionEncoder(feature_dim=64).to(DEVICE)
print(f"Vision encoder parameters: "
      f"{sum(p.numel() for p in vision_enc.parameters()):,}")

### 4.4 — The Language Encoder

For language, we use a simple embedding layer that maps each color name to a learned feature vector. In the real $\pi_0$, this would be a full language model (like PaLM or Gemma). For our task, since the vocabulary is just three color names, an embedding suffices.

In [ ]:
class LanguageEncoder(nn.Module):
    """Embedding-based encoder for color instructions."""

    def __init__(self, vocab_size=3, feature_dim=64):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, feature_dim)

    def forward(self, instruction_ids):
        # instruction_ids: (B,) integer tensor
        return self.embedding(instruction_ids)  # (B, feature_dim)

# Vocabulary mapping
VOCAB = {name: i for i, name in enumerate(COLOR_NAMES)}
print(f"Vocabulary: {VOCAB}")

lang_enc = LanguageEncoder(vocab_size=len(VOCAB),
                           feature_dim=64).to(DEVICE)
print(f"Language encoder parameters: "
      f"{sum(p.numel() for p in lang_enc.parameters()):,}")

### 4.5 — The Fusion Module

Now we combine vision and language features. We concatenate them and pass through a small MLP to produce a single fused representation. This fused vector will condition our flow matching decoder.

In [ ]:
class FusionModule(nn.Module):
    """Fuse vision and language features via concatenation + MLP."""

    def __init__(self, vision_dim=64, lang_dim=64, fused_dim=128):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(vision_dim + lang_dim, fused_dim),
            nn.ReLU(),
            nn.Linear(fused_dim, fused_dim),
            nn.ReLU(),
        )

    def forward(self, vision_feat, lang_feat):
        combined = torch.cat([vision_feat, lang_feat], dim=-1)
        return self.mlp(combined)  # (B, fused_dim)

fusion = FusionModule(fused_dim=128).to(DEVICE)
print(f"Fusion module parameters: "
      f"{sum(p.numel() for p in fusion.parameters()):,}")

Let us visualize the fused features for different instructions to verify the fusion module differentiates between color commands (even before training, the features should differ due to random initialization).

In [ ]:
# Encode a small batch to visualize
sample_imgs = torch.tensor(
    images[:6], dtype=torch.float32
).permute(0, 3, 1, 2).to(DEVICE) / 255.0
sample_ids = torch.tensor(
    [VOCAB[c] for c in instructions[:6]], dtype=torch.long
).to(DEVICE)

with torch.no_grad():
    v_feat = vision_enc(sample_imgs)
    l_feat = lang_enc(sample_ids)
    fused = fusion(v_feat, l_feat)

fig, axes = plt.subplots(2, 3, figsize=(12, 5))
for idx in range(6):
    row, col = idx // 3, idx % 3
    axes[row, col].bar(range(fused.shape[1]),
                       fused[idx].cpu().numpy(), width=1.0,
                       alpha=0.7)
    axes[row, col].set_title(f'"{instructions[idx]}" features')
    axes[row, col].set_xlim(0, fused.shape[1])
plt.suptitle('Fused Vision + Language Features (Before Training)',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

### 4.6 — The Flow Matching Action Decoder

This is the heart of our VLA. The decoder takes:
1. **Fused features** (from vision + language)
2. **A noisy action trajectory** $x_t$ (the current state in the flow)
3. **A timestep** $t$ (where we are in the flow from noise to data)

And outputs the predicted velocity field $v_\theta(x_t, t)$ — the direction to move from noise toward a valid action trajectory.

In [ ]:
class FlowMatchingDecoder(nn.Module):
    """Predicts velocity field for flow matching.

    Takes fused conditioning features, noisy trajectory,
    and flow timestep. Outputs predicted velocity.
    """

    def __init__(self, fused_dim=128, traj_len=TRAJ_LEN,
                 action_dim=ACTION_DIM, hidden_dim=256):
        super().__init__()
        self.traj_len = traj_len
        self.action_dim = action_dim
        traj_flat = traj_len * action_dim  # flattened trajectory

        # Timestep embedding
        self.time_embed = nn.Sequential(
            nn.Linear(1, 64),
            nn.SiLU(),
            nn.Linear(64, 64),
        )

        # Main network
        self.net = nn.Sequential(
            nn.Linear(fused_dim + traj_flat + 64, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, traj_flat),
        )

    def forward(self, fused_feat, x_t, t):
        """
        Args:
            fused_feat: (B, fused_dim) conditioning
            x_t: (B, traj_len, action_dim) noisy trajectory
            t: (B, 1) flow timestep in [0, 1]
        Returns:
            v_theta: (B, traj_len, action_dim) predicted velocity
        """
        B = fused_feat.shape[0]
        x_flat = x_t.view(B, -1)
        t_emb = self.time_embed(t)
        inp = torch.cat([fused_feat, x_flat, t_emb], dim=-1)
        out = self.net(inp)
        return out.view(B, self.traj_len, self.action_dim)

In [ ]:
decoder = FlowMatchingDecoder(fused_dim=128).to(DEVICE)
print(f"Flow matching decoder parameters: "
      f"{sum(p.numel() for p in decoder.parameters()):,}")

# Total model parameters
total_params = sum(
    sum(p.numel() for p in m.parameters())
    for m in [vision_enc, lang_enc, fusion, decoder]
)
print(f"\nTotal VLA parameters: {total_params:,}")
print("(A real VLA like pi-zero has billions — ours is tiny "
      "but captures the same architecture!)")

### 4.7 — The Flow Matching Training Loop

Now let us implement the training procedure. Recall the key steps:

1. Sample a batch of (image, instruction, target trajectory)
2. Sample random noise $x_0 \sim \mathcal{N}(0, I)$
3. Sample random flow time $t \sim U(0, 1)$
4. Compute interpolation $x_t = (1-t) \cdot x_0 + t \cdot x_1$
5. Predict velocity $v_\theta(x_t, t)$
6. Loss = $\|v_\theta - (x_1 - x_0)\|^2$

In [ ]:
def flow_matching_loss(decoder, fused_feat, target_traj):
    """Compute the flow matching training loss.

    Args:
        decoder: FlowMatchingDecoder
        fused_feat: (B, fused_dim) conditioning features
        target_traj: (B, T, 2) ground truth trajectory (x_1)
    Returns:
        loss: scalar tensor
    """
    B = target_traj.shape[0]

    # Step 1: Sample noise x_0
    x_0 = torch.randn_like(target_traj)

    # Step 2: Sample random timestep t ~ U(0, 1)
    t = torch.rand(B, 1, device=target_traj.device)

    # Step 3: Interpolate x_t = (1-t)*x_0 + t*x_1
    t_expand = t.unsqueeze(-1)  # (B, 1, 1)
    x_t = (1 - t_expand) * x_0 + t_expand * target_traj

    # Step 4: True velocity is (x_1 - x_0)
    true_velocity = target_traj - x_0

    # Step 5: Predict velocity
    pred_velocity = decoder(fused_feat, x_t, t)

    # Step 6: MSE loss
    loss = F.mse_loss(pred_velocity, true_velocity)
    return loss

### 4.8 — Flow Matching Inference

At inference time, we start from pure noise and iteratively follow the learned velocity field to generate the action trajectory. This is an ODE (Ordinary Differential Equation) integration using Euler steps.

In [ ]:
@torch.no_grad()
def flow_matching_inference(decoder, fused_feat,
                            n_steps=20):
    """Generate action trajectory via flow matching inference.

    Start from noise, integrate the velocity field using
    Euler steps from t=0 to t=1.

    Args:
        decoder: trained FlowMatchingDecoder
        fused_feat: (B, fused_dim) conditioning
        n_steps: number of Euler integration steps
    Returns:
        x_t: (B, TRAJ_LEN, ACTION_DIM) generated trajectory
    """
    B = fused_feat.shape[0]
    dt = 1.0 / n_steps

    # Start from noise
    x_t = torch.randn(B, TRAJ_LEN, ACTION_DIM,
                       device=fused_feat.device)

    # Euler integration
    for step in range(n_steps):
        t = torch.full((B, 1), step * dt,
                       device=fused_feat.device)
        velocity = decoder(fused_feat, x_t, t)
        x_t = x_t + dt * velocity

    return x_t

Let us visualize the flow field before training. Since the model is randomly initialized, the velocity vectors will point in random directions — but after training, they will consistently point from noise toward valid trajectories.

In [ ]:
# Visualize flow field (pre-training): 2D slice
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

# Create a grid of points in 2D (first action step)
grid_pts = 10
xx, yy = np.meshgrid(np.linspace(-2, 2, grid_pts),
                      np.linspace(-2, 2, grid_pts))
grid = np.stack([xx.ravel(), yy.ravel()], axis=-1)

# Create dummy fused features (single instruction)
dummy_fused = torch.zeros(grid_pts**2, 128, device=DEVICE)
# Create dummy trajectories (only first step varies)
dummy_traj = torch.zeros(grid_pts**2, TRAJ_LEN, ACTION_DIM,
                         device=DEVICE)
dummy_traj[:, 0, :] = torch.tensor(grid, dtype=torch.float32,
                                    device=DEVICE)
dummy_t = torch.full((grid_pts**2, 1), 0.5, device=DEVICE)

with torch.no_grad():
    vel = decoder(dummy_fused, dummy_traj, dummy_t)
vel_2d = vel[:, 0, :].cpu().numpy()

ax.quiver(grid[:, 0], grid[:, 1],
          vel_2d[:, 0], vel_2d[:, 1],
          color='cyan', alpha=0.7)
ax.set_title('Flow Field (Before Training) — Random Directions',
             fontsize=12)
ax.set_xlabel('dx')
ax.set_ylabel('dy')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Section 5 — 🔧 Your Turn (TODO Exercises)

Now it is your turn to implement two critical components. These exercises will solidify your understanding of flow matching.

### TODO 1: Implement the Flow Matching Interpolation and Loss

Fill in the missing parts of the function below. You need to:
1. Compute the interpolation $x_t = (1 - t) \cdot x_0 + t \cdot x_1$
2. Compute the target velocity $u_t = x_1 - x_0$
3. Compute the MSE loss between predicted and target velocity

In [ ]:
def flow_matching_loss_todo(decoder, fused_feat, target_traj):
    """TODO: Implement flow matching loss.

    Args:
        decoder: FlowMatchingDecoder
        fused_feat: (B, fused_dim) conditioning features
        target_traj: (B, T, 2) ground truth trajectory (x_1)
    Returns:
        loss: scalar tensor
    """
    B = target_traj.shape[0]
    x_0 = torch.randn_like(target_traj)
    t = torch.rand(B, 1, device=target_traj.device)
    t_expand = t.unsqueeze(-1)  # (B, 1, 1) for broadcasting

    # ---- YOUR CODE HERE ----
    # Step 1: Compute x_t (interpolation between x_0 and target)
    x_t = None  # TODO

    # Step 2: Compute the target velocity
    target_velocity = None  # TODO

    # Step 3: Get model's predicted velocity
    pred_velocity = decoder(fused_feat, x_t, t)

    # Step 4: Compute MSE loss
    loss = None  # TODO
    # ---- END YOUR CODE ----

    return loss

Run the verification cell below to check your implementation.

In [ ]:
# Verification for TODO 1
print("Verifying TODO 1: Flow Matching Loss")
print("=" * 45)

# Create small test tensors
_B, _T, _D = 4, TRAJ_LEN, ACTION_DIM
_fused = torch.randn(_B, 128, device=DEVICE)
_traj = torch.randn(_B, _T, _D, device=DEVICE)

try:
    _loss = flow_matching_loss_todo(decoder, _fused, _traj)
    assert _loss is not None, "Loss is None — did you forget to compute it?"
    assert _loss.shape == torch.Size([]), \
        f"Loss should be scalar, got shape {_loss.shape}"
    assert _loss.item() > 0, "Loss should be positive"
    assert _loss.requires_grad, "Loss should require gradients"
    print(f"Loss value: {_loss.item():.4f}")
    print("All checks passed!")
except Exception as e:
    print(f"Error: {e}")
    print("Hint: x_t = (1-t)*x_0 + t*x_1, velocity = x_1 - x_0")

### TODO 2: Implement the Inference Loop

Now implement the inference procedure. Starting from random noise, take $K$ Euler steps following the learned velocity field to generate the final action trajectory.

In [ ]:
@torch.no_grad()
def flow_matching_inference_todo(decoder, fused_feat,
                                 n_steps=20):
    """TODO: Generate trajectory via flow matching inference.

    Start from noise x_0 ~ N(0, I), then for each step k:
      t = k / n_steps
      velocity = decoder(fused_feat, x_t, t)
      x_t = x_t + dt * velocity

    Args:
        decoder: trained FlowMatchingDecoder
        fused_feat: (B, fused_dim) conditioning
        n_steps: number of Euler integration steps
    Returns:
        x_t: (B, TRAJ_LEN, ACTION_DIM) generated trajectory
    """
    B = fused_feat.shape[0]
    dt = 1.0 / n_steps

    # ---- YOUR CODE HERE ----
    # Step 1: Start from random noise
    x_t = None  # TODO

    # Step 2: Euler integration loop (n_steps iterations)
    # For each step k from 0 to n_steps-1:
    #   - compute t = k * dt
    #   - predict velocity from decoder
    #   - update x_t = x_t + dt * velocity
    pass  # TODO: replace with your loop
    # ---- END YOUR CODE ----

    return x_t

Run the verification cell below. Since the model is not trained yet, the generated trajectory will be random — but we can verify the shapes and that the function runs.

In [ ]:
# Verification for TODO 2
print("Verifying TODO 2: Flow Matching Inference")
print("=" * 45)

_fused = torch.randn(4, 128, device=DEVICE)
try:
    _gen_traj = flow_matching_inference_todo(decoder, _fused,
                                             n_steps=10)
    assert _gen_traj is not None, \
        "Output is None — did you forget to return x_t?"
    assert _gen_traj.shape == (4, TRAJ_LEN, ACTION_DIM), \
        f"Expected shape (4, {TRAJ_LEN}, {ACTION_DIM}), " \
        f"got {_gen_traj.shape}"
    assert not torch.isnan(_gen_traj).any(), \
        "Output contains NaN values"
    print(f"Output shape: {_gen_traj.shape}")
    print(f"Output range: [{_gen_traj.min():.3f}, "
          f"{_gen_traj.max():.3f}]")
    print("All checks passed!")
except Exception as e:
    print(f"Error: {e}")
    print("Hint: initialize x_t with torch.randn, then loop "
          "with Euler steps")

## Section 6 — Putting It All Together

Let us now assemble the full VLA pipeline into a single class that combines all four components: vision encoder, language encoder, fusion module, and flow matching decoder.

In [ ]:
class VisionLanguageActionModel(nn.Module):
    """Complete VLA: image + text → action trajectory.

    Architecture:
        Image → VisionEncoder → vision_feat
        Text  → LanguageEncoder → lang_feat
        (vision_feat, lang_feat) → FusionModule → fused
        (fused, noise, t) → FlowMatchingDecoder → velocity
    """

    def __init__(self):
        super().__init__()
        self.vision_enc = VisionEncoder(feature_dim=64)
        self.lang_enc = LanguageEncoder(
            vocab_size=len(VOCAB), feature_dim=64)
        self.fusion = FusionModule(fused_dim=128)
        self.decoder = FlowMatchingDecoder(fused_dim=128)

    def get_fused_features(self, images, instruction_ids):
        """Encode image + text and fuse."""
        v_feat = self.vision_enc(images)
        l_feat = self.lang_enc(instruction_ids)
        return self.fusion(v_feat, l_feat)

    def compute_loss(self, images, instruction_ids,
                     target_traj):
        """Full forward pass + flow matching loss."""
        fused = self.get_fused_features(images, instruction_ids)
        return flow_matching_loss(self.decoder, fused,
                                  target_traj)

    @torch.no_grad()
    def generate(self, images, instruction_ids, n_steps=20):
        """Generate action trajectory from image + text."""
        fused = self.get_fused_features(images, instruction_ids)
        return flow_matching_inference(self.decoder, fused,
                                       n_steps)

# Instantiate the full model
vla_model = VisionLanguageActionModel().to(DEVICE)
total = sum(p.numel() for p in vla_model.parameters())
print(f"Complete VLA model: {total:,} parameters")

## Section 7 — Training and Results

Now let us train our VLA on the 2D reaching task. We will train for enough epochs to see the flow matching loss converge, then test generalization to unseen target positions.

In [ ]:
# Prepare data tensors
train_images = torch.tensor(
    images, dtype=torch.float32
).permute(0, 3, 1, 2) / 255.0  # (N, 3, H, W)
train_ids = torch.tensor(
    [VOCAB[c] for c in instructions], dtype=torch.long)
train_trajs = torch.tensor(trajectories, dtype=torch.float32)

print(f"Training images: {train_images.shape}")
print(f"Training instructions: {train_ids.shape}")
print(f"Training trajectories: {train_trajs.shape}")

In [ ]:
# Training hyperparameters
BATCH_SIZE = 128
LEARNING_RATE = 3e-4
N_EPOCHS = 40

optimizer = torch.optim.Adam(vla_model.parameters(),
                             lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=N_EPOCHS)
n_batches = len(train_images) // BATCH_SIZE

# Training loop
losses = []
vla_model.train()

for epoch in range(N_EPOCHS):
    epoch_loss = 0.0
    perm = torch.randperm(len(train_images))

    for batch_idx in range(n_batches):
        idx = perm[batch_idx * BATCH_SIZE:
                   (batch_idx + 1) * BATCH_SIZE]
        imgs = train_images[idx].to(DEVICE)
        ids = train_ids[idx].to(DEVICE)
        trajs = train_trajs[idx].to(DEVICE)

        loss = vla_model.compute_loss(imgs, ids, trajs)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    scheduler.step()
    avg_loss = epoch_loss / n_batches
    losses.append(avg_loss)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:3d}/{N_EPOCHS} | "
              f"Loss: {avg_loss:.6f} | "
              f"LR: {scheduler.get_last_lr()[0]:.6f}")

print("\nTraining complete!")

Let us plot the training loss curve. We expect the flow matching loss to decrease smoothly as the model learns to predict the correct velocity field.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(losses, 'b-', linewidth=2)
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Flow Matching Loss', fontsize=12)
ax.set_title('Training Loss: VLA with Flow Matching',
             fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_yscale('log')
plt.tight_layout()
plt.show()
print(f"Final loss: {losses[-1]:.6f}")

### Testing on Novel Positions

The real test of our VLA: can it generalize to target positions it has never seen during training? Let us create new scenes with unseen target locations and color combinations.

In [ ]:
vla_model.eval()

# Generate test scenes with novel positions
test_targets = [
    (np.array([0.2, 0.85]), 'red'),
    (np.array([0.8, 0.4]),  'blue'),
    (np.array([0.5, 0.7]),  'green'),
    (np.array([0.7, 0.85]), 'red'),
    (np.array([0.3, 0.5]),  'blue'),
    (np.array([0.6, 0.55]), 'green'),
]

theta1, theta2 = np.pi / 3, np.pi / 4
_, end_eff = forward_kinematics(theta1, theta2)

test_imgs = []
test_ids = []
for pos, color in test_targets:
    img = render_scene(pos, color, theta1, theta2)
    test_imgs.append(img)
    test_ids.append(VOCAB[color])

test_imgs_t = torch.tensor(
    np.stack(test_imgs), dtype=torch.float32
).permute(0, 3, 1, 2).to(DEVICE) / 255.0
test_ids_t = torch.tensor(test_ids, dtype=torch.long).to(DEVICE)

In [ ]:
# Generate trajectories
gen_trajs = vla_model.generate(test_imgs_t, test_ids_t,
                                n_steps=30)
gen_trajs_np = gen_trajs.cpu().numpy()

# Also compute ground truth for comparison
gt_trajs = []
for pos, _ in test_targets:
    gt = generate_trajectory(end_eff, pos)
    gt_trajs.append(gt)
gt_trajs_np = np.stack(gt_trajs)

print("Generated trajectory shapes:", gen_trajs_np.shape)
print("Ground truth trajectory shapes:", gt_trajs_np.shape)

In [ ]:
# Visualize: generated vs ground truth trajectories
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for idx in range(6):
    row, col = idx // 3, idx % 3
    ax = axes[row, col]
    ax.imshow(test_imgs[idx])

    h, w = test_imgs[idx].shape[:2]

    # Ground truth trajectory (green)
    gt_pos = np.cumsum(gt_trajs_np[idx], axis=0) + end_eff
    gt_px = gt_pos[:, 0] * w
    gt_py = (1 - gt_pos[:, 1]) * h
    ax.plot(gt_px, gt_py, 'g-', linewidth=2, alpha=0.8,
            label='Ground Truth')

    # Generated trajectory (yellow)
    gen_pos = np.cumsum(gen_trajs_np[idx], axis=0) + end_eff
    gen_px = gen_pos[:, 0] * w
    gen_py = (1 - gen_pos[:, 1]) * h
    ax.plot(gen_px, gen_py, 'y--', linewidth=2, alpha=0.9,
            label='Generated')

    color_name = test_targets[idx][1]
    ax.set_title(f'"reach the {color_name} circle"',
                 fontsize=11)
    ax.axis('off')
    if idx == 0:
        ax.legend(loc='lower right', fontsize=8)

plt.suptitle('Generated vs Ground Truth Trajectories '
             '(Novel Positions)',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Flow Matching vs Direct Regression

Let us now compare flow matching with a simpler baseline: **direct regression**, where the model directly predicts the trajectory without the flow process. This will demonstrate why flow matching produces smoother, higher-quality trajectories.

In [ ]:
class DirectRegressionDecoder(nn.Module):
    """Baseline: directly predicts trajectory (no flow)."""

    def __init__(self, fused_dim=128, traj_len=TRAJ_LEN,
                 action_dim=ACTION_DIM, hidden_dim=256):
        super().__init__()
        traj_flat = traj_len * action_dim
        self.net = nn.Sequential(
            nn.Linear(fused_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, traj_flat),
        )
        self.traj_len = traj_len
        self.action_dim = action_dim

    def forward(self, fused_feat):
        out = self.net(fused_feat)
        return out.view(-1, self.traj_len, self.action_dim)

In [ ]:
# Train the direct regression baseline
baseline_decoder = DirectRegressionDecoder(fused_dim=128
                                           ).to(DEVICE)

# Reuse vision/language/fusion from vla_model (frozen)
baseline_opt = torch.optim.Adam(baseline_decoder.parameters(),
                                lr=3e-4)
baseline_losses = []

for epoch in range(N_EPOCHS):
    epoch_loss = 0.0
    perm = torch.randperm(len(train_images))

    for batch_idx in range(n_batches):
        idx = perm[batch_idx * BATCH_SIZE:
                   (batch_idx + 1) * BATCH_SIZE]
        imgs = train_images[idx].to(DEVICE)
        ids = train_ids[idx].to(DEVICE)
        trajs = train_trajs[idx].to(DEVICE)

        with torch.no_grad():
            fused = vla_model.get_fused_features(imgs, ids)
        pred = baseline_decoder(fused)
        loss = F.mse_loss(pred, trajs)

        baseline_opt.zero_grad()
        loss.backward()
        baseline_opt.step()

        epoch_loss += loss.item()

    avg = epoch_loss / n_batches
    baseline_losses.append(avg)

    if (epoch + 1) % 10 == 0:
        print(f"Baseline Epoch {epoch+1:3d}/{N_EPOCHS} | "
              f"Loss: {avg:.6f}")

print("Baseline training complete!")

In [ ]:
# Compare trajectories: flow matching vs direct regression
baseline_decoder.eval()

with torch.no_grad():
    fused_test = vla_model.get_fused_features(test_imgs_t,
                                               test_ids_t)
    baseline_trajs = baseline_decoder(fused_test).cpu().numpy()

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for idx in range(6):
    row, col = idx // 3, idx % 3
    ax = axes[row, col]

    # Ground truth
    gt_pos = np.cumsum(gt_trajs_np[idx], axis=0) + end_eff
    ax.plot(gt_pos[:, 0], gt_pos[:, 1], 'g-o', linewidth=2,
            markersize=3, label='Ground Truth', alpha=0.7)

    # Flow matching (ours)
    fm_pos = np.cumsum(gen_trajs_np[idx], axis=0) + end_eff
    ax.plot(fm_pos[:, 0], fm_pos[:, 1], 'b-s', linewidth=2,
            markersize=3, label='Flow Matching', alpha=0.7)

    # Direct regression (baseline)
    dr_pos = np.cumsum(baseline_trajs[idx], axis=0) + end_eff
    ax.plot(dr_pos[:, 0], dr_pos[:, 1], 'r-^', linewidth=2,
            markersize=3, label='Direct Regression', alpha=0.7)

    # Target
    target_pos = test_targets[idx][0]
    ax.plot(*target_pos, '*', color='gold', markersize=15,
            zorder=5, label='Target')

    ax.set_title(f'"reach the {test_targets[idx][1]} circle"')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.2)
    if idx == 0:
        ax.legend(fontsize=7, loc='lower right')

plt.suptitle('Flow Matching vs Direct Regression',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

Let us also compare the smoothness of the trajectories quantitatively. We measure smoothness as the variance of the velocity (lower = smoother).

In [ ]:
def trajectory_smoothness(traj_deltas):
    """Measure smoothness as variance of step-to-step changes.
    Lower = smoother.
    """
    # Second differences (jerk)
    jerk = np.diff(traj_deltas, axis=0)
    return np.mean(jerk ** 2)

print("Trajectory Smoothness (lower = smoother)")
print("=" * 45)

fm_smooth = np.mean([trajectory_smoothness(gen_trajs_np[i])
                     for i in range(6)])
dr_smooth = np.mean([trajectory_smoothness(baseline_trajs[i])
                     for i in range(6)])
gt_smooth = np.mean([trajectory_smoothness(gt_trajs_np[i])
                     for i in range(6)])

print(f"Ground Truth:       {gt_smooth:.6f}")
print(f"Flow Matching:      {fm_smooth:.6f}")
print(f"Direct Regression:  {dr_smooth:.6f}")
print(f"\nFlow matching produces "
      f"{dr_smooth/fm_smooth:.1f}x smoother trajectories!")

### Visualizing the Learned Flow Field

Now that the model is trained, let us revisit the flow field visualization. This time, the velocity arrows should consistently point from noise toward valid action trajectories. Compare this with the random arrows we saw before training.

In [ ]:
# Visualize learned flow field at different timesteps
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

t_values = [0.0, 0.5, 0.9]
grid_pts = 12
xx, yy = np.meshgrid(np.linspace(-2, 2, grid_pts),
                      np.linspace(-2, 2, grid_pts))
grid = np.stack([xx.ravel(), yy.ravel()], axis=-1)
n_pts = grid_pts ** 2

# Use a real fused feature (red target top-right)
with torch.no_grad():
    sample_fused = vla_model.get_fused_features(
        test_imgs_t[:1].repeat(n_pts, 1, 1, 1),
        test_ids_t[:1].repeat(n_pts))

for ax_idx, t_val in enumerate(t_values):
    dummy_traj = torch.zeros(n_pts, TRAJ_LEN, ACTION_DIM,
                             device=DEVICE)
    dummy_traj[:, 0, :] = torch.tensor(
        grid, dtype=torch.float32, device=DEVICE)
    t_tensor = torch.full((n_pts, 1), t_val, device=DEVICE)

    with torch.no_grad():
        vel = vla_model.decoder(sample_fused, dummy_traj,
                                t_tensor)
    vel_2d = vel[:, 0, :].cpu().numpy()

    axes[ax_idx].quiver(grid[:, 0], grid[:, 1],
                        vel_2d[:, 0], vel_2d[:, 1],
                        color='cyan', alpha=0.8)
    axes[ax_idx].set_title(f'Learned Flow Field (t={t_val})',
                           fontsize=12)
    axes[ax_idx].set_xlabel('dx')
    axes[ax_idx].set_ylabel('dy')
    axes[ax_idx].set_aspect('equal')
    axes[ax_idx].grid(True, alpha=0.3)

plt.suptitle('Velocity Field at Different Flow Timesteps '
             '(After Training)',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

## Section 8 — 🎯 Final Output

Let us bring everything together in a stunning final visualization. We will show a 2x3 grid of different language commands, each producing a different generated trajectory overlaid on the scene. Then we will animate one trajectory showing the "robot" moving step by step.

In [ ]:
# Final showcase: 2x3 grid of different commands
showcase = [
    (np.array([0.25, 0.80]), 'red',   'reach the red circle'),
    (np.array([0.75, 0.45]), 'blue',  'reach the blue circle'),
    (np.array([0.50, 0.90]), 'green', 'reach the green circle'),
    (np.array([0.15, 0.55]), 'red',   'reach the red circle'),
    (np.array([0.85, 0.75]), 'blue',  'reach the blue circle'),
    (np.array([0.40, 0.65]), 'green', 'reach the green circle'),
]

show_imgs = []
show_ids = []
for pos, color, _ in showcase:
    img = render_scene(pos, color, theta1, theta2)
    show_imgs.append(img)
    show_ids.append(VOCAB[color])

show_imgs_t = torch.tensor(
    np.stack(show_imgs), dtype=torch.float32
).permute(0, 3, 1, 2).to(DEVICE) / 255.0
show_ids_t = torch.tensor(show_ids, dtype=torch.long
                          ).to(DEVICE)

with torch.no_grad():
    show_trajs = vla_model.generate(
        show_imgs_t, show_ids_t, n_steps=30
    ).cpu().numpy()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 11))

for idx in range(6):
    row, col = idx // 3, idx % 3
    ax = axes[row, col]
    ax.imshow(show_imgs[idx])

    h, w = show_imgs[idx].shape[:2]
    gen_pos = np.cumsum(show_trajs[idx], axis=0) + end_eff
    px = gen_pos[:, 0] * w
    py = (1 - gen_pos[:, 1]) * h

    # Color gradient along trajectory
    points = np.array([px, py]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    norm = plt.Normalize(0, TRAJ_LEN)
    lc = LineCollection(segments, cmap='plasma', norm=norm,
                        linewidth=3, alpha=0.9)
    lc.set_array(np.arange(TRAJ_LEN))
    ax.add_collection(lc)

    # Start and end markers
    ax.plot(px[0], py[0], 'wo', markersize=8, zorder=5)
    ax.plot(px[-1], py[-1], 'w*', markersize=12, zorder=5)

    ax.set_title(f'"{showcase[idx][2]}"', fontsize=11,
                 fontweight='bold')
    ax.axis('off')

plt.suptitle('VLA in Action: One Model — See, Understand, Act',
             fontsize=15, fontweight='bold',
             color='#2d3436')
plt.tight_layout()
plt.show()

Now let us animate one trajectory to see the "robot" moving step by step toward its target. This is the closest we get to watching our VLA control a real robot.

In [ ]:
# Animate one trajectory
anim_idx = 0
anim_img = show_imgs[anim_idx]
anim_traj = show_trajs[anim_idx]
anim_pos = np.cumsum(anim_traj, axis=0) + end_eff

h, w = anim_img.shape[:2]
anim_px = anim_pos[:, 0] * w
anim_py = (1 - anim_pos[:, 1]) * h

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(anim_img)
ax.axis('off')
ax.set_title(f'"{showcase[anim_idx][2]}"', fontsize=12)

line, = ax.plot([], [], 'y-', linewidth=2, alpha=0.8)
dot, = ax.plot([], [], 'yo', markersize=10, zorder=5)

def init():
    line.set_data([], [])
    dot.set_data([], [])
    return line, dot

def animate(frame):
    f = min(frame, len(anim_px) - 1)
    line.set_data(anim_px[:f+1], anim_py[:f+1])
    dot.set_data([anim_px[f]], [anim_py[f]])
    return line, dot

anim = animation.FuncAnimation(
    fig, animate, init_func=init,
    frames=TRAJ_LEN + 5, interval=150, blit=True)
plt.close(fig)

# Display in notebook
HTML(anim.to_jshtml())

In [ ]:
# Congratulations message
print("=" * 55)
print("  CONGRATULATIONS!")
print("=" * 55)
print()
print("  You have built a Vision-Language-Action model")
print("  that takes an image + language command and")
print("  generates smooth robot action trajectories")
print("  using flow matching!")
print()
print("  What you implemented:")
print("  - Vision Encoder (CNN)")
print("  - Language Encoder (Embeddings)")
print("  - Feature Fusion Module")
print("  - Flow Matching Action Decoder")
print("  - Training with velocity field prediction")
print("  - Inference via ODE integration")
print()
print("  This is the same architecture (simplified)")
print("  behind Physical Intelligence's pi-zero, which")
print("  controls real robots from language instructions.")
print()
print("  You have completed all 6 notebooks in the")
print("  World Action Models series!")
print("=" * 55)

## Section 9 — Reflection

### Three Questions to Think About

1. **Scalability:** Our VLA uses a tiny CNN and a simple embedding for language. Real systems like $\pi_0$ use pre-trained vision transformers (SigLIP) and language models (Gemma). How would scaling up these components change the model's capabilities? What new behaviors might emerge?

2. **Multimodality of actions:** In our 2D task, there is essentially one good trajectory to each target. But in the real world, there are often multiple valid ways to perform a task (e.g., reaching around an obstacle from the left or right). How does flow matching handle this multimodality compared to direct regression? (Hint: think about what happens when you average two different valid trajectories.)

3. **From simulation to reality:** Our model was trained entirely in a simple 2D simulation. V-JEPA 2 showed that pre-training on internet video can dramatically reduce the amount of real-robot data needed. Why might watching videos of the world (even without any robot in them) help a robot learn to manipulate objects?

### Optional Challenges

**Challenge 1: Add Obstacle Avoidance**

Add a static obstacle (a gray circle) to the scene. Modify the training data generation to produce trajectories that curve around the obstacle. Then test whether the VLA learns to avoid obstacles when generating trajectories. You will need to modify `render_scene` and `generate_trajectory`.

**Challenge 2: Three-Link Arm**

Extend the robot arm from 2 links to 3 links. This increases the complexity of forward kinematics and makes the reaching task more interesting — there are now multiple joint configurations that reach the same target (redundancy). Implement the new kinematics and observe how the model handles this added complexity.

### Series Recap — World Action Models

Over the course of six notebooks, we have journeyed from basic world models to the cutting edge of embodied AI:

1. **World Models** — learned to imagine the future in latent space
2. **Latent Dynamics** — built models that predict state transitions without pixel reconstruction
3. **Planning in Imagination** — used world models to plan actions by simulating outcomes
4. **Video Prediction** — extended world models to predict future video frames
5. **Self-Supervised Physical Understanding** — learned physics from observation alone (JEPA-style)
6. **Vision-Language-Action** — the grand convergence: see, understand, act

The trajectory of this field is clear: we are moving toward **general-purpose embodied agents** that can understand any instruction, perceive any environment, and act smoothly in the physical world. The models are getting larger, the training data is scaling from simulation to internet-scale video, and the gap between digital intelligence and physical capability is closing rapidly.

This is the frontier. And now you have the foundations to understand — and contribute to — what comes next.

See you next time!